conda_tensorflow_p36

In [1]:
import subprocess, json

In [2]:
def add_class(method):
    return "public class Test {\n"+method+"\n}"

In [3]:
def remove_class(parse_tree):
    return {'classbody': parse_tree['compilationUnit'][0]['typeDeclaration'][1]['classDeclaration'][2]['classBody'][1:-1]}

In [4]:
def parse(code):
    code = add_class(code)
    tree = subprocess.check_output(['java', '-jar', 'mavenproject1-1.0-SNAPSHOT-jar-with-dependencies.jar', code])
    tree = json.loads(tree)
    tree = remove_class(tree)
    return tree

In [5]:
def tree2code(tree, indentation):
    if 'type' in tree:
        code = tree['text'] + ' '
        if tree['text'] in [';', '{', '}']:
            code += '\n'
            if tree['text'] == '{':
                indentation += 1
            elif tree['text'] == '}':
                indentation -= 1
            code += '\t'*indentation
        return code, indentation
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        code = ""
        for child in children:
            t, indentation = tree2code(child, indentation)
            code += t
        return code, indentation

In [40]:
code = """@ Override public int run Command ( boolean merge Error Into Output , String ... commands ) throws IO Exception , Interrupted Exception { return run Command ( merge Error Into Output , new Array List < String > ( Arrays . as List ( commands ) ) ) ; }"""
tree = parse(code)
tree

{'classbody': [{'classBodyDeclaration': [{'modifier': [{'classOrInterfaceModifier': [{'annotation': [{'type': 101,
           'text': '@'},
          {'annotationName': [{'qualifiedName': [{'type': 100,
               'text': 'Override'}]}]}]}]}]},
    {'modifier': [{'classOrInterfaceModifier': [{'type': 35,
         'text': 'public'}]}]},
    {'memberDeclaration': []}]},
  {'classBodyDeclaration': [{'memberDeclaration': [{'type': 27,
       'text': 'int'}]}]},
  {'classBodyDeclaration': [{'memberDeclaration': [{'methodDeclaration': [{'typeSpec': [{'classOrInterfaceType': [{'type': 100,
             'text': 'run'}]}]},
        {'type': 100, 'text': 'Command'},
        {'formalParameters': [{'type': 57, 'text': '('},
          {'formalParameterList': [{'formalParameter': [{'typeSpec': [{'primitiveType': [{'type': 3,
                   'text': 'boolean'}]}]},
              {'variableDeclaratorId': [{'type': 100,
                 'text': 'merge'}]}]}]}]}]}]}]},
  {'classBodyDeclaration': 

In [44]:
print(tree2code(tree,-1)[0])

@ Override public int run Command ( boolean merge Error Into Output , String ... commands ) throws IO Exception , Interrupted <missing ';'> Exception { 
return run <missing ';'> Command ( merge <missing ';'> Error Into <missing ';'> Output , new Array <missing ';'> List < String > ( Arrays . as <missing ';'> List ( commands ) ) ) ; 
} 



In [ ]:
# add class
# parse
# build dictionary
# Build Numto compress
# compress
# tokenize
# remove class tokens

# Building Dictionary

In [6]:
abstract_code_dict = {}
compression_number = 0
freq_dict = {}

In [ ]:
import datetime
start = datetime.datetime.now()

path = '../code_summarization/train/code.original'
print("=====[", path,"]=====")
i = 0
try:
    with open(path) as data:
        for raw_code in data:
            if i%10000 == 0: print("[", i, "]")
            
            tree = parse(raw_code)
            (code, leaf_count, id_count, var_list) = build_n_compress(tree, True)
            i+=1
except:
    print("Line:",i,"is corrupted. Cannot build dictionary.")

for key in list(abstract_code_dict):
    if abstract_code_dict[key]['count'] < 2:
        del abstract_code_dict[key]

end = datetime.datetime.now()
print(end-start)
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)
end = datetime.datetime.now()
print("Dictionary building complete in - ",end-start)

In [15]:
# Use glob to find all files
# Or just list the 6 directories manually
paths = [
    '../code_summarization/dev/code.original',
    '../code_summarization/test/code.original',
    '../code_summarization/train/code.original'
]
paths

['../code_summarization/dev/code.original',
 '../code_summarization/test/code.original',
 '../code_summarization/train/code.original']

In [ ]:
## build freq_dict


In [ ]:
## add func_names from abstract_code_dict, set count: 0

for i in range in len(abstract_code_dict):
    func = '$F'+str(i)
    freq_dict[func] = 0

In [ ]:
## initialize vocab_size
vocab_size = 50000

In [ ]:
import re

num_to_compress = 0
for i in range(len(abstract_code_keys)):
    func_name = abstract_code_dict[abstract_code_keys[i]]['compressed_name']
    code = abstract_code_keys[i]
    count = abstract_code_dict[abstract_code_keys[i]]['count']
    most_frequent_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
    
    if count >= freq_dict[most_frequent_list[vocab_size]]:
        token_list = [x for x in code.split() if x[:3] != '$id']
        for token in token_list:
            freq_dict[token] -= count
        freq_dict[func_name] += count
    else:
        num_to_compress = i
        print("Number to Compress:",i)
        break
most_frequent_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)